#### Student Name: Mai Ngo
#### Course Name and Number: DSC 578 Neural Network and Deep Learning - SEC 701
#### Homework5 - Modified Network code
#### Date: 10/29/2023

In [1]:
import random
import json
import numpy as np
import pandas as pd
random.seed(17)

### Definitions of cost functions (as function classes)

In [2]:
class QuadraticCost(object):
    @staticmethod
    def fn(actualOutput, targetOutput):
        '''Compute quadratic cost using L2 (Euclidean) norm'''
        return 0.5*np.linalg.norm(targetOutput-actualOutput)**2

    @staticmethod
    def derivative(actualOutput, targetOutput):
        '''Return the first derivative of the function with respect to actual output.'''
        return -(targetOutput-actualOutput)

class CrossEntropyCost(object):
    @staticmethod
    def fn(actualOutput, targetOutput):
        ''' Computes Cross Entropy cost. Ensure numerical stability: like log(0), NaN'''
        return np.sum(np.nan_to_num(-targetOutput*np.log(actualOutput)-(1-targetOutput)*np.log(1-actualOutput)))

    @staticmethod
    def derivative (actualOutput, targetOutput):
        '''Return the first derivative of the function with respect to actual output.'''
        return (actualOutput-targetOutput)/(actualOutput*(1-actualOutput))
              
class LogLikelihood(object):
    @staticmethod
    def fn(actualOutput, targetOutput):
        ''' Computes Log Likelihood cost. Ensure numerical stability: like log(0), NaN'''
        return np.sum(np.nan_to_num(-targetOutput*np.log(actualOutput)))

    @staticmethod
    def derivative(actualOutput, targetOutput):
        '''Return the first derivative of the function with respect to actual output.'''
        LL_dev = np.zeros_like(actualOutput)
        targetOutput_index = np.argmax(targetOutput)  # Get the index of the target category
        LL_dev[targetOutput_index] = -1 / actualOutput[targetOutput_index]
        return LL_dev

### Definitions of activation functions (as function classes)

In [3]:
class Sigmoid(object):
    @staticmethod
    def fn(z):
        '''Compute the sigmoid activation for a given input.'''
        return 1.0/(1.0+np.exp(-z))

    @classmethod
    def derivative(cls,z):
        '''Derivative of the sigmoid function.'''
        return cls.fn(z)*(1-cls.fn(z))

class Softmax(object):
    @staticmethod
    def fn(z):
        '''The softmax of vector z, aka. activation outputs @ final layer. Parameter z is an array of shape (len(z), 1).'''
        expZ = np.exp(z)
        return expZ/expZ.sum()
       
    @classmethod
    def derivative(cls,z):
        '''Derivative of the softmax.'''
        softmaxVector = cls.fn(z) 
        return np.diagflat(softmaxVector) - np.dot(softmaxVector, softmaxVector.T)

class Tanh(object):
    @staticmethod
    def fn(z):
        '''The tanh function.'''
        return (np.exp(z) - np.exp(-z))/(np.exp(z) + np.exp(-z))

    @classmethod
    def derivative(cls,z):
        '''Derivative of the tanh function.'''
        return 1 - (cls.fn(z)**2)

class ReLU(object):
    @staticmethod
    def fn(z):
        '''The ReLU function.'''
        return np.maximum(0, z)

    @classmethod
    def derivative(cls,z):
        """Derivative of the ReLU function."""
        return np.where(z > 0, 1, 0)

class LeakyReLU(object):
    def __init__(self, alpha):
        self.alpha = alpha
      
    def fn(self, z):
        '''The LeakyReLU function.'''
        return np.maximum(self.alpha*z, z)
  
    def derivative(self, z):
        '''Derivative of the LeakyReLU function.'''
        return np.where(z < 0, self.alpha, 1)

### The main Network class

In [4]:
def vectorizeTarget(n, targetOutput):
    '''Return an array of shape (n,1) with a 1.0 in the target position, zeroes elsewhere.  
    The parameter target is assumed to be an array of size 1, and the 0th item is the target position (1).'''
    targetArray = np.zeros((n, 1))
    targetArray[int(targetOutput[0])] = 1.0
    return targetArray

In [5]:
def loadCSV(fileName, inputSize, targetSize, seedNum=17):
    ''' Load the data from a csv file.  Target (y) is already in the one-hot-vector notation (binary representation).
        inputSize: # of Xs | targetSize: # of Ys. 
        Output as a list with each element contains a pair of Xs and Ys (formatted as one column vector respectively). 
        Total # of element = Total # of instances.'''

    data = pd.read_csv(fileName, header=None)
    #Set the random seed if specified to shuffle, for reproducibility. Otherwise no shuffling.
    if seedNum:
        data = data.sample(frac=1, random_state=seedNum)

    #Separate the X and Y parts
    X = data[data.columns[:inputSize]].values.tolist()
    Y = data[data.columns[-targetSize:]].values.tolist()

    #Combine the parts for each instance and put all in a list. 
    #For each instance, zip(X,Y) pairs input feature vector and its original corresponding target value vector.
    dataset = [(np.reshape(x, (inputSize, 1)), np.reshape(y, (targetSize, 1)))
               for x, y in zip(X, Y)]
    return dataset

In [6]:
class Network(object):
    '''class Network for stochastic gradient descent and hyperparameters tuning.'''
    
    def __init__(self, networkSize, dropoutPercent=0.0, dropoutMask=None, regularization=None, lmbda=0.0):
        '''Initialize network's number of layers, and neuron count within each layer. Biases and weights are initialized randomly.'''
        self.numLayers = len(networkSize)
        self.networkSize = networkSize
        self.dropoutPercent = dropoutPercent
        self.dropoutMask = dropoutMask
        self.regularization = regularization
        self.lmbda = lmbda
        self.default_weightInitializer()

    def default_weightInitializer(self):
        '''Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1, over the square root of the number of
        weights connecting to the same neuron.'''
        
        #Biases start from the 2nd layer. neuronNum_l1 = # of neurons in the layer.
        self.biases = [np.random.randn(neuronNum_l1, 1) for neuronNum_l1 in self.networkSize[1:]]
        
        #Weights return as an array size (# neurons next layer, # neurons current layer).
        self.weights = [np.random.randn(neuronNum_l1, neuronNum_l)/np.sqrt(neuronNum_l)
                        for neuronNum_l, neuronNum_l1 in zip(self.networkSize[:-1], self.networkSize[1:])]

    def set_modelParameters(self, cost=CrossEntropyCost, actHidden=Sigmoid, actOutput=None):
        '''Set model parameters.'''
        self.cost = cost
        self.actHidden = actHidden
        if actOutput is None:
            self.actOutput = self.actHidden
        else:
            self.actOutput = actOutput
        
        #Change based on request.
        if self.actOutput == Tanh:
            print('Error: Tanh cannot be used for output layer.  Changing to Sigmoid!')
            self.actOutput = Sigmoid

    def set_compileParameters(self, regularization=None, lmbda=0.0, dropoutPercent=0.0, dropoutMask=None):
        '''Set compiler parameters.'''
        self.regularization = regularization
        self.lmbda = lmbda
        self.dropoutPercent = dropoutPercent
        self.dropoutMask = dropoutMask

    def SGD(self, trainData, epochs, mini_batchSize, eta, testData=None, regularization=None, lmbda=0.0):
        '''Train the neural network using mini-batch stochastic gradient descent.
        trainData: a list of tuples (X,Y). testData: optional.
        epochs: # of times the network will be trained on the entire trainData.'''
        
        trainNum = len(trainData) 
        if testData:
            testNum = len(testData)

        trainCost, trainAccuracy = [], []
        testCost, testAccuracy = [], []

        for epoch in range(epochs):
            #Get a list of mini batches with individual mini_batchSize.
            miniBatches = [trainData[m : m + mini_batchSize]
                for m in range(0, trainNum, mini_batchSize)]
            
            for miniBatch in miniBatches:
                if self.dropoutPercent != 0.0: #Custom dropout mask.
                    if self.dropoutMask is None: 
                        self.dropoutMask = [np.zeros(size) for size in self.networkSize[1:-1]]
                    for hiddenLayer, dropoutMask in zip(self.networkSize[1:-1], self.dropoutMask):
                        
                        #Number of neurons to keep.
                        retainCount = round(hiddenLayer * (1 - self.dropoutPercent)) 
                        #Get index of neuron to keep using random.sample.
                        retainIndex = random.sample(range(hiddenLayer), retainCount)
                        for index in range(hiddenLayer):
                            if index in retainIndex: #Scale active neuron per request.
                                dropoutMask[index] = 1.0 / (1.0 - self.dropoutPercent)
                
                self.update_miniBatch(miniBatch, eta, trainNum)
                self.dropoutMask = None #Reset for next mini batch.
                
            #Evaluation for both training and evaluation datasets.
            cost_trainData = self.totalCost(trainData)
            trainCost.append(cost_trainData)

            accuracy_trainData = self.evaluateAccuracy(trainData)
            trainAccuracy.append(accuracy_trainData)

            if testData:
                cost_testData= self.totalCost(testData)
                testCost.append(cost_testData)
                accuracy_testData = self.evaluateAccuracy(testData)
                testAccuracy.append(accuracy_testData)

        print ("Training {} epochs complete.\n".format(epochs))
        return trainCost, trainAccuracy, testCost, testAccuracy

    def customSign(self, w): #Was debate between np.sign and custom function.
        '''Sign function.'''
        return np.where(w > 0, 1, np.where(w == 0, 0, -1))

    def update_miniBatch(self, miniBatch, eta, n):
        '''Update the network's weights and biases to a single mini batch.
        miniBatch: a list of tuples (X, Y), eta: learning rate.'''
        
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        for X, Y in miniBatch:
            backprop_nabla_b, backprop_nabla_w = self.backprop(X, Y)
            for i in range(len(nabla_b)): #Iterate over each layer.
                nabla_b[i] += backprop_nabla_b[i]
                nabla_w[i] += backprop_nabla_w[i]
        
        if self.regularization == 'L1':
            self.weights = [w - (eta * (self.lmbda/n) * np.sign(w)) -((eta/len(miniBatch)) * nw) for w, nw in zip(self.weights, nabla_w)]
        elif self.regularization == 'L2':
            self.weights = [(1-eta * (self.lmbda/n)) * w-(eta/len(miniBatch)) * nw for w, nw in zip(self.weights, nabla_w)]
        else: #If no regularization used, not apply. 
            self.weights = [w - (eta / len(miniBatch)) * nw for w, nw in zip(self.weights, nabla_w)]
        
        self.biases = [b-(eta/len(miniBatch)) * nb for b, nb in zip(self.biases, nabla_b)]
                
    def backprop(self, X, Y):
        '''Calculate the gradients with respect to the network's parameters (weights and biases).
        Return a tuple (nabla_b, nabla_w).'''
        
        #Create two lists of arrays with the same shapes as the network's biases and weights.
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        #Forward pass: start with initial instanceInput X
        #Activations list contains the activations of each layer progressed through the network.
        activation = X
        activations = [X] #List to store all the activations, layer by layer.
        zList = [] #List to store all the z vectors, layer by layer.
        
        i = 0
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            z = np.dot(w, activation)+b
            zList.append(z)
            activation = (self.actHidden).fn(z)
            if self.dropoutMask is not None:
                dropoutMask = self.dropoutMask[i]  # Use the corresponding dropout mask
                activation *= dropoutMask.reshape(-1, 1)
            activations.append(activation)
            i += 1
            
        #For output layer.
        b = self.biases[-1]
        w = self.weights[-1]
        z = np.dot(w, activation) + b
        zList.append(z)
        activation = (self.actOutput).fn(z)
        activations.append(activation)
        
        #Backward pass
        ##Cost and activation functions are parameterized now.
        ##Call the activation function of the output layer with z.
        activationPrime = (self.actOutput).derivative(zList[-1]) #Derivative with respect to activation output z.
        costPrime = (self.cost).derivative(activations[-1], Y) #Derivative with respect to actual ouput. 

        #Separate case for Softmax
        if self.actOutput == Softmax:
            delta = np.dot(activationPrime, costPrime)
        else:
            delta = costPrime * activationPrime
        
        #Error gradient (finalDelta), nabla weight and bias @ final output.
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        
        #l=1 means the last layer of neurons, l=2 is the second-last layer, and so on.  
        for l in range(2, self.numLayers):
            z = zList[-l]
            activation_prime = (self.actHidden).derivative(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * activation_prime
            if self.dropoutMask is not None:
                dropoutMask = self.dropoutMask[-l + 1]
                delta *= dropoutMask.reshape(-1, 1)
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluateAccuracy(self, data):
        '''Get total number of correct predictions.
        For each instance, a tuple is created with two elements:
        1. The index of the maximum value in actual output.
        2. The index of the maximum value in target output.
        Counts how many of these pairs have matching indices, indicating correct predictions.'''
        results = [(np.argmax(self.feedForward(instanceInput)), np.argmax(targetOutput)) for (instanceInput, targetOutput) in data]
        return sum(int(actualOutput == targetOutput) for (actualOutput, targetOutput) in results)
        
    def totalCost(self, data):
        '''Return the total cost for input dataset.'''
        cost = 0.0
        for instanceInput, targetOutput in data:
            actualOutput = self.feedForward(instanceInput)
            cost += self.cost.fn(actualOutput, targetOutput)/len(data)
        
        penaltyValue = 0.0
        if self.regularization == 'L1':
            penaltyValue += (self.lmbda/len(data)) * np.sum([np.sum(np.abs(w)) for w in self.weights])
    
        elif self.regularization == 'L2':
            penaltyValue += 0.5*(self.lmbda/len(data)) * sum(np.linalg.norm(w)**2 for w in self.weights)
        cost += penaltyValue               
        return cost

    def feedForward(self, activationOutput):
        '''Return network activation output given 'activationOutput' as input.
        Use for evaluation; not during training/backpropagation.'''
        for b, w in zip(self.biases[:-1], self.weights[:-1]):
            activationOutput = (self.actHidden).fn(np.dot(w, activationOutput) + b)
        b = self.biases[-1]
        w = self.weights[-1]
        activationOutput = (self.actOutput).fn(np.dot(w,activationOutput) + b)
        return activationOutput

    def saveNetwork(self, fileName):
        '''Save the neural network to a JSON file.'''
        data = {"sizes": self.networkSize, "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases], "cost": str(self.cost.__name__)}
        outFile = open(fileName, "w")
        json.dump(data, outFile)
        outFile.close()

    @classmethod
    def loadNetwork(cls, fileName):
        '''Load a neural network from a JSON file and return an instance of Network.'''
        try:
            with open(fileName, "r") as inFile:
                data = json.load(inFile)
            network = cls(data["sizes"])
            network.weights = [np.array(w) for w in data["weights"]]
            network.biases = [np.array(b) for b in data["biases"]]
            return network
        except (FileNotFoundError, json.JSONDecodeError) as e:
            print(f"Error loading network from {fileName}: {e}")
            return None  